In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
data = pd.read_csv('housing.csv')
data

FileNotFoundError: [Errno 2] No such file or directory: 'housing.csv'

In [ ]:
data.info()

: 

In [ ]:
data.dropna(inplace=True)

: 

In [ ]:
data.info()

: 

In [ ]:
data.dropna(inplace=True)

: 

In [ ]:
data.info()

: 

In [ ]:
from sklearn.model_selection import train_test_split

: 

In [ ]:
X = data.drop(['median_house_value'], axis=1)
Y = data['median_house_value']

: 

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

: 

In [ ]:
train_data = X_train.join(Y_train)

: 

In [ ]:
train_data

: 

In [ ]:
train_data.hist(figsize=(15,8))

: 

# **Data Pre-processing**

In [ ]:
 train_data.corr()

: 

In [ ]:
plt.figure(figsize=(15,8))
sns.heatmap(train_data.corr(), annot=True, cmap="YlGnBu")

: 

In [ ]:
train_data['total_rooms'] = np.log(train_data['total_rooms'] + 1)
train_data['total_bedrooms'] = np.log(train_data['total_bedrooms'] + 1)
train_data['population'] = np.log(train_data['population'] + 1)
train_data['households'] = np.log(train_data['households'] + 1)

: 

In [ ]:
train_data.hist(figsize=(15,8))

: 

In [ ]:
train_data = train_data.join(pd.get_dummies(train_data.ocean_proximity)).drop(['ocean_proximity'], axis=1)

: 

In [ ]:
train_data

: 

In [ ]:
plt.figure(figsize=(15,8))
sns.heatmap(train_data.corr(), annot=True, cmap="YlGnBu")

: 

In [ ]:
plt.figure(figsize=(15,8))
sns.scatterplot(x='latitude', y='longitude', data=train_data, hue='median_house_value', palette='coolwarm')

: 

In [ ]:
train_data['bedroom_ratio'] = train_data['total_bedrooms'] / train_data['total_rooms']
train_data['household_rooms'] = train_data['total_rooms'] / train_data['households']

: 

In [ ]:
plt.figure(figsize=(15,8))
sns.heatmap(train_data.corr(), annot=True, cmap="YlGnBu")

: 

# **Training the Model**

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

: 

In [ ]:
scaler = StandardScaler()

: 

In [ ]:
X_train, Y_train = train_data.drop(['median_house_value'], axis=1), train_data['median_house_value']
X_train_s = scaler.fit_transform(X_train)

reg = LinearRegression()

reg.fit(X_train, Y_train)

: 

In [ ]:
test_data = X_test.join(Y_test)

test_data['total_rooms'] = np.log(test_data['total_rooms'] + 1)
test_data['total_bedrooms'] = np.log(test_data['total_bedrooms'] + 1)
test_data['population'] = np.log(test_data['population'] + 1)
test_data['households'] = np.log(test_data['households'] + 1)

test_data = test_data.join(pd.get_dummies(test_data.ocean_proximity)).drop(['ocean_proximity'], axis=1)

test_data['bedroom_ratio'] = test_data['total_bedrooms'] / test_data['total_rooms']
test_data['household_rooms'] = test_data['total_rooms'] / test_data['households']

: 

In [ ]:
test_data

: 

In [ ]:
X_test, Y_test = test_data.drop(['median_house_value'], axis=1), test_data['median_house_value']


: 

In [ ]:
X_test_s = scaler.transform(X_test)

: 

In [ ]:
reg.score(X_test, Y_test)


: 

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor()

forest.fit(X_train_s, Y_train)

: 

In [ ]:
forest.score(X_test_s, Y_test)


: 

In [ ]:
from sklearn.model_selection import GridSearchCV


: 

In [ ]:
forest = RandomForestRegressor()

param_grid = {
    "n_estimators": [100, 200, 300],
    "min_samples_split": [2,4],
    "max_depth": [None,4, 8]
}

grid_search = GridSearchCV(forest, param_grid, cv=5, scoring='neg_mean_squared_error',return_train_score=True)

grid_search.fit(X_train_s, Y_train)

: 

In [ ]:
grid_search.best_estimator_

: 